In [5]:
# !pip install requests
# !pip install beautifulsoup4
!pip install pandas

In [10]:

from bs4 import BeautifulSoup
import pandas as pd
import zipfile
import requests
import string
import pdb
import io
import os

os.mkdir('Amazon/')
base_repository = "Amazon/"

## Retriving the data

In [2]:
def get_file_name(url):
  url = url.split('/')
  url = url[len(url)-1].strip()
  return url.translate(str.maketrans(" ", " ", string.punctuation))

def save_file(url, file_name):
  data, buf = requests.get(url), io.BytesIO()
  with zipfile.ZipFile(f"{base_repository}{file_name}.zip", "w") as file:
    jsonl = '.'.join([file_name.split('json')[0], 'jsonl'])
    file.writestr(jsonl, data.content)

def retrieve_list_of_url():
  website = requests.get('https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023')
  if website.status_code == 200:
    soup = BeautifulSoup(website.text, 'html.parser')

    def has_class_but_no_id(tag):
      return tag.has_attr('rel') and tag.has_attr('download') and tag.has_attr('href')
    a_links = soup.findAll(has_class_but_no_id)
    return [a.get('href') for a in a_links]
  else:
    print("Generating Url Failed")
    return []

def download_data_and_store():
  org_data = pd.DataFrame({}, index=['rating', 'title', 'text', 'asin', 'parent_asin', 'user_id', 'timestamp', 'verified_purchase', 'helpful_vote'])
  for i in retrieve_list_of_url():
    file_name = get_file_name(i)
    org_name = f"{base_repository}{file_name}"
    save_file(i, file_name)
    with zipfile.ZipFile(f"{org_name}.zip", 'r') as file:
      file.extractall(org_name)
    # os.remove(f"{org_name}.zip")
    jsonl = '.'.join([file_name.split('json')[0], 'jsonl'])
    data = pd.read_json(f"{org_name}{jsonl}", lines=True)
    org_data.join(data)
  return org_data


In [32]:
# data = download_data_and_store()
# data = pd.read_json(f"Amazon/AllBrautyjsonlgz/AllBeauty.jsonl", lines=True)
os.getcwd()
with zipfile.ZipFile(f"Amazon/AllBeautyjsonlgz.zip", 'r') as zip_file:
  first_file = zip_file.namelist()[0]
  with zip_file.open(first_file, 'r') as file:
      print(file.read())
#     content = file.read()
#     content = content.decode('utf-8')
#     print(type(content[5]))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

